In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [17]:
X_test.shape

(2000, 13)

In [18]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [19]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5912 - accuracy: 0.6694
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4779 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4555 - accuracy: 0.7991
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4420 - accuracy: 0.8085
Epoch 6/100
250/250 [==============================] - 0s 990us/step - loss: 0.4303 - accuracy: 0.8165
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.8185
Epoch 8/100
250/250 [==============================] - 0s 980us/step - loss: 0.4165 - accuracy: 0.8186
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4098 - accuracy: 0.8221
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4043 - ac

250/250 [==============================] - 0s 1ms/step - loss: 0.3350 - accuracy: 0.8636
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3346 - accuracy: 0.8629
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3323 - accuracy: 0.8633
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3329 - accuracy: 0.8650
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3326 - accuracy: 0.8637
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3323 - accuracy: 0.8624
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3320 - accuracy: 0.8635
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3324 - accuracy: 0.8629
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3326 - accuracy: 0.8626
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3320 - accuracy: 

# undersampling

In [20]:
count_class_0, count_class_1 = df1.Exited.value_counts()
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [21]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [22]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [23]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [24]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 1ms/step - loss: 0.6873 - accuracy: 0.5707
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6653 - accuracy: 0.6346
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6436 - accuracy: 0.6453
Epoch 4/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6247 - accuracy: 0.6622
Epoch 5/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6111 - accuracy: 0.6747
Epoch 6/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5968 - accuracy: 0.6824
Epoch 7/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5865 - accuracy: 0.6932
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5718 - accuracy: 0.7030
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5571 - accuracy: 0.7159
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5415 - accura

102/102 [==============================] - 0s 1ms/step - loss: 0.4670 - accuracy: 0.7745
Epoch 82/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4662 - accuracy: 0.7766
Epoch 83/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4658 - accuracy: 0.7748
Epoch 84/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.7720
Epoch 85/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4659 - accuracy: 0.7766
Epoch 86/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4653 - accuracy: 0.7775
Epoch 87/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4651 - accuracy: 0.7757
Epoch 88/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4642 - accuracy: 0.7763
Epoch 89/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4660 - accuracy: 0.7736
Epoch 90/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4630 - accuracy: 

# Oversampling

In [25]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [26]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [27]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6422 - accuracy: 0.6306
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6057 - accuracy: 0.6699
Epoch 3/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5869 - accuracy: 0.6902
Epoch 4/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5764 - accuracy: 0.7020
Epoch 5/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5592 - accuracy: 0.7149
Epoch 6/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5332 - accuracy: 0.7401
Epoch 7/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5141 - accuracy: 0.7523
Epoch 8/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5008 - accuracy: 0.7618
Epoch 9/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4931 - accuracy: 0.7613
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4870 - accura

399/399 [==============================] - 1s 1ms/step - loss: 0.4452 - accuracy: 0.7870
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4452 - accuracy: 0.7892
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4451 - accuracy: 0.7911
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4453 - accuracy: 0.7889
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4450 - accuracy: 0.7920
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4452 - accuracy: 0.7882
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4450 - accuracy: 0.7889
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4458 - accuracy: 0.7904
Epoch 89/100
399/399 [==============================] - 1s 2ms/step - loss: 0.4446 - accuracy: 0.7874
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4436 - accuracy: 

# smote

In [28]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [29]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [31]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6609 - accuracy: 0.6084
Epoch 2/100
399/399 [==============================] - 0s 1ms/step - loss: 0.6172 - accuracy: 0.6664
Epoch 3/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5898 - accuracy: 0.6940
Epoch 4/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5725 - accuracy: 0.7078
Epoch 5/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5590 - accuracy: 0.7146
Epoch 6/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5403 - accuracy: 0.7276
Epoch 7/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5186 - accuracy: 0.7467
Epoch 8/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5006 - accuracy: 0.7555
Epoch 9/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4873 - accuracy: 0.7657
Epoch 10/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4764 - accura

399/399 [==============================] - 0s 1ms/step - loss: 0.4259 - accuracy: 0.7990
Epoch 82/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4261 - accuracy: 0.7983
Epoch 83/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4256 - accuracy: 0.7993
Epoch 84/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4249 - accuracy: 0.8009
Epoch 85/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.7991
Epoch 86/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4259 - accuracy: 0.7989
Epoch 87/100
399/399 [==============================] - 0s 1000us/step - loss: 0.4251 - accuracy: 0.8000
Epoch 88/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4249 - accuracy: 0.7994
Epoch 89/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4245 - accuracy: 0.8004
Epoch 90/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4252 - accurac

# Ensemble

In [32]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [33]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [35]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [36]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [37]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [38]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [39]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6880 - accuracy: 0.5283
Epoch 2/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6589 - accuracy: 0.6250
Epoch 3/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6398 - accuracy: 0.6467
Epoch 4/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6279 - accuracy: 0.6550
Epoch 5/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6190 - accuracy: 0.6662
Epoch 6/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6089 - accuracy: 0.6781
Epoch 7/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6004 - accuracy: 0.6861
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5955 - accuracy: 0.6867
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5897 - accuracy: 0.6944
Epoch 10/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5848 - accuracy: 0.6950
Epoch 11/

98/98 [==============================] - 0s 1ms/step - loss: 0.4495 - accuracy: 0.7875
Epoch 84/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4494 - accuracy: 0.7846
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4485 - accuracy: 0.7843
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4485 - accuracy: 0.7856
Epoch 87/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4480 - accuracy: 0.7843
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4495 - accuracy: 0.7805
Epoch 89/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.7830
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4474 - accuracy: 0.7821
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4464 - accuracy: 0.7866
Epoch 92/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4489 - accuracy: 0.7834
Epoch 93/100


In [40]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6877 - accuracy: 0.5718
Epoch 2/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6559 - accuracy: 0.6237
Epoch 3/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6345 - accuracy: 0.6522
Epoch 4/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6230 - accuracy: 0.6611
Epoch 5/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6140 - accuracy: 0.6726
Epoch 6/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6081 - accuracy: 0.6854
Epoch 7/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6014 - accuracy: 0.6851
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5937 - accuracy: 0.6867
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5868 - accuracy: 0.6966
Epoch 10/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5813 - accuracy: 0.6960
Epoch 11/

98/98 [==============================] - 0s 980us/step - loss: 0.4603 - accuracy: 0.7766
Epoch 83/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4604 - accuracy: 0.7779
Epoch 84/100
98/98 [==============================] - 0s 991us/step - loss: 0.4596 - accuracy: 0.7766
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4596 - accuracy: 0.7805
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4596 - accuracy: 0.7808
Epoch 87/100
98/98 [==============================] - 0s 996us/step - loss: 0.4591 - accuracy: 0.7789
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4584 - accuracy: 0.7808
Epoch 89/100
98/98 [==============================] - 0s 997us/step - loss: 0.4596 - accuracy: 0.7741
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4592 - accuracy: 0.7811
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4589 - accuracy: 0.7792
Epoch

In [41]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 920us/step - loss: 0.6845 - accuracy: 0.5599
Epoch 2/100
87/87 [==============================] - 0s 931us/step - loss: 0.6535 - accuracy: 0.6083
Epoch 3/100
87/87 [==============================] - 0s 980us/step - loss: 0.6281 - accuracy: 0.6567
Epoch 4/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6076 - accuracy: 0.6783
Epoch 5/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5931 - accuracy: 0.6931
Epoch 6/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5826 - accuracy: 0.7011
Epoch 7/100
87/87 [==============================] - 0s 936us/step - loss: 0.5710 - accuracy: 0.7076
Epoch 8/100
87/87 [==============================] - 0s 954us/step - loss: 0.5596 - accuracy: 0.7166
Epoch 9/100
87/87 [==============================] - 0s 972us/step - loss: 0.5518 - accuracy: 0.7271
Epoch 10/100
87/87 [==============================] - 0s 979us/step - loss: 0.5400 - accuracy: 0.

87/87 [==============================] - 0s 1ms/step - loss: 0.4499 - accuracy: 0.7888
Epoch 83/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4503 - accuracy: 0.7870
Epoch 84/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4520 - accuracy: 0.7827
Epoch 85/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4513 - accuracy: 0.7863
Epoch 86/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4497 - accuracy: 0.7845
Epoch 87/100
87/87 [==============================] - 0s 976us/step - loss: 0.4504 - accuracy: 0.7816
Epoch 88/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4493 - accuracy: 0.7816
Epoch 89/100
87/87 [==============================] - 0s 999us/step - loss: 0.4495 - accuracy: 0.7881
Epoch 90/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.7877
Epoch 91/100
87/87 [==============================] - 0s 987us/step - loss: 0.4485 - accuracy: 0.7801
Epoch 9

In [42]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [43]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.75      0.84      1593
           1       0.45      0.80      0.58       407

    accuracy                           0.76      2000
   macro avg       0.70      0.78      0.71      2000
weighted avg       0.84      0.76      0.78      2000

